In [1]:
import stl.mesh
import numpy
import trimesh
import random

import meshutil
import examples

In [2]:
#mesh = examples.ram_horn()
#mesh = examples.twist()
#mesh = examples.twist_nonlinear()

In [65]:
def gen_twisted_boundary(count=4, dx0=2, dz=0.2, ang=0.1):
    b = numpy.array([
        [0, 0, 0],
        [1, 0, 0],
        [1, 0, 1],
        [0, 0, 1],
    ], dtype=numpy.float64) - [0.5, 0, 0.5]
    # this function almost needs some sort of 'continuous'
    # property about it - that it is not tied to an integral
    # number of iterations of a transform, but can smoothly
    # generate it at any resolution.
    # but... can I do that? do all transforms accomodate that?
    # some should (rotation, translation), for others it makes
    # no sense (mirroring)
    #
    # Generate 'seed' transformations:
    xfs = [meshutil.Transform().translate(dx0, 0, 0).rotate([0,1,0], numpy.pi * 2 * i / count)
           for i in range(count)]
    # (we'll increment the transforms in xfs as we go)
    while True:
        xfs_new = []
        bs = []
        for i, xf in enumerate(xfs):
            # Generate a boundary from running transform:
            b_i = xf.apply_to(b)
            bs.append(b_i)
            # Increment transform i:
            xf2 = xf.rotate([0,1,0], ang)
            xfs_new.append(xf2)
        xfs = xfs_new
        yield bs

In [66]:
# This is to see how well it works to compose generators:
def gen_inc_y(gen, dy=0.1):
    xf = meshutil.Transform()
    for bs in gen:
        bs2 = [xf.apply_to(b) for b in bs]
        yield bs2
        xf = xf.translate(0, dy, 0)

In [67]:
# and to string together boundaries from a generator:
def gen2mesh(gen, count):
    # Get first list of boundaries:
    bs_last = next(gen)
    # TODO: Begin and end with close_boundary_simple
    mesh = meshutil.FaceVertexMesh.Empty()
    for i,bs_cur in enumerate(gen):
        for j,b in enumerate(bs_cur):
            m = meshutil.join_boundary_simple(bs_last[j], b)
            mesh = mesh.concat(m)
        bs_last = bs_cur
        if i > count:
            break
    return mesh

In [68]:
gen = gen_inc_y(gen_twisted_boundary())
mesh = gen2mesh(gen, 100)
# TODO: Fix winding order

In [28]:
# what even is this?
"""
dx0=2
frames=100
b = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [1, 0, 1],
    [0, 0, 1],
], dtype=numpy.float64) - [0.5, 0.5, 0]
ang = -numpy.pi*2 / frames
# negative because of winding order annoyance
mesh = meshutil.FaceVertexMesh.Empty()
xf = meshutil.Transform() \
    .translate(dx0, 0, 0)
b0 = xf.apply_to(b)
for layer in range(frames):
    b_sub0 = xf.apply_to(b)
    incr = meshutil.Transform().rotate([0,0,1], ang)
    b_sub1 = xf.compose(incr).apply_to(b)
    m = meshutil.join_boundary_simple(b_sub0, b_sub1)
    mesh = mesh.concat(m)
    xf = xf.compose(incr)
"""

In [3]:
#mesh = torus_xy()

In [70]:
# Work around some annoying-ass trimesh/threejs bug:
wtf = meshutil.Transform().scale(0.1).translate(5,0,0)
center = meshutil.Transform().translate(-0.5, -0.5, -0.5)
base = meshutil.cube(open_xz=False).transform(center)
base = base.transform(wtf)
# to enable:
mesh = mesh.concat(base)

mesh_fname = "twist.stl"
mesh.to_stl_mesh().save(mesh_fname)

# TODO: Just use the mesh data directly (no sense in saving & re-loading)
m = trimesh.load_mesh(mesh_fname)
#m.show()
scene = trimesh.Scene([m])
scene.show()

face_normals didn't match triangles, ignoring!
